In [1]:
!nvidia-smi

Fri Mar 15 02:08:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:1B.0 Off |                    0 |
| N/A   27C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import platform,socket,re,uuid,json,psutil,logging

def getSystemInfo():
    try:
        info={}
        info['platform']=platform.system()
        info['platform-release']=platform.release()
        info['platform-version']=platform.version()
        info['architecture']=platform.machine()
        info['hostname']=socket.gethostname()
        info['ip-address']=socket.gethostbyname(socket.gethostname())
        info['mac-address']=':'.join(re.findall('..', '%012x' % uuid.getnode()))
        info['processor']=platform.processor()
        info['ram']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB"
        return json.dumps(info)
    except Exception as e:
        logging.exception(e)

json.loads(getSystemInfo())

{'platform': 'Linux',
 'platform-release': '6.2.0-1017-aws',
 'platform-version': '#17~22.04.1-Ubuntu SMP Fri Nov 17 21:07:13 UTC 2023',
 'architecture': 'x86_64',
 'hostname': 'ip-10-25-205-144',
 'ip-address': '10.25.205.144',
 'mac-address': '02:3f:27:fa:f4:5d',
 'processor': 'x86_64',
 'ram': '187 GB'}

In [1]:
import torch
print(torch.version.cuda)
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f'Number of GPUs available: {gpu_count}')
    for i in range(gpu_count):
        print(f'GPU {i}: {torch.cuda.get_device_name(i)}')
else:
    print('No GPU available.')

12.1
Number of GPUs available: 4
GPU 0: Tesla T4
GPU 1: Tesla T4
GPU 2: Tesla T4
GPU 3: Tesla T4


In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

2024-03-15 02:09:20.663610: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 02:09:21.949279: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 02:09:23.864453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [38]:
# import os
# os.environ['PATH'] += ":/home/ziyuwang/.local/bin"

In [ ]:
# os.environ['ENV_VAR'] = '/home/ziyuwang/.local/bin'

In [2]:
# !pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade
# # install git-fls for pushing model and logs to the hugging face hub
# !sudo apt-get install git-lfs --yes

Defaulting to user installation because normal site-packages is not writeable
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# !pip3 install 'urllib3<2'

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from datasets import load_dataset, DatasetDict

# # Load dataset from the hub
dataset = load_dataset("AlexWang99/MixSub")


In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Abstract', 'Highlights'],
        num_rows: 16294
    })
})


In [6]:
shuffled_dataset = dataset['train'].shuffle(seed=42) # Use a fixed seed for reproducibility

# Split the shuffled dataset into train, validation, and test sets (80%, 10%, 10%)
dataset_split = shuffled_dataset.train_test_split(test_size=0.2)
test_validation_split = dataset_split['test'].train_test_split(test_size=0.5)

train_set = dataset_split['train']
validation_set = test_validation_split['test']
test_set = test_validation_split['train']

# Add "highlight: " prefix to the 'abstract' field of each set
def add_prefix(examples):
    examples['Abstract'] = 'Highlight: ' + examples['Abstract']
    return examples

train_set = train_set.map(add_prefix)
validation_set = validation_set.map(add_prefix)
test_set = test_set.map(add_prefix)
dataset = DatasetDict({
    'train': train_set,
    'test': test_set
})

Map:   0%|          | 0/13035 [00:00<?, ? examples/s]

Map:   0%|          | 0/1630 [00:00<?, ? examples/s]

Map:   0%|          | 0/1629 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Abstract"], truncation=True), batched=True, remove_columns=["Abstract", "Highlights"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Highlights"], truncation=True), batched=True, remove_columns=["Abstract", "Highlights"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/14664 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/14664 [00:00<?, ? examples/s]

Max target length: 421


In [9]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["Abstract"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["Highlights"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["Abstract", "Highlights"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/13035 [00:00<?, ? examples/s]

Map:   0%|          | 0/1629 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [10]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [11]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

2024-03-15 04:59:41.695411: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 04:59:41.749775: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 04:59:42.725906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /home/ziyuwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [15]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
dataset_id="AlexWang99/MixSub"
# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [27]:
# !pip3 install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/3a/49/12fc5188602c68a789a0fdaee63d176a71ad5c1e34d25aeb8554abe46089/torchvision-0.17.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/a6/57/ccebdda4db80e384166c70d8645fa998637051b3b19aca1fd8de80602afb/torchaudio-2.2.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/2c/df/5810707da6f2fd4be57f0cc417987c0fa16a2eecf0b1b71f82ea555dc619/torch-2.2.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for nvidia-cuda-nvrtc-cu12==12.1.105 from https://files.pythonhosted.org/packages/b6/9f/c64c03f49d6fbc56196664d05dba14e3a561038a81a638eeb47f4d4cfd48/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtai

In [16]:
trainer.train()

/home/ziyuwang/.local/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.381100,2.097150,21.391600,8.172800,18.024000,19.899500,19.000000
2,2.238900,2.070955,21.778000,8.456500,18.329100,20.247400,19.000000
3,2.195400,2.059263,21.780400,8.503100,18.348600,20.231000,19.000000
4,2.156800,2.053983,21.665700,8.406900,18.173800,20.062200,19.000000
5,2.135400,2.053181,21.652700,8.378200,18.193100,20.046100,19.000000


/home/ziyuwang/.local/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/ziyuwang/.local/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ziyuwang/.local/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/ziyuwang/.local/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dim

TrainOutput(global_step=4075, training_loss=2.2133076186852954, metrics={'train_runtime': 7620.8135, 'train_samples_per_second': 8.552, 'train_steps_per_second': 0.535, 'total_flos': 4.46290604457984e+16, 'train_loss': 2.2133076186852954, 'epoch': 5.0})

In [17]:
trainer.evaluate()

/home/ziyuwang/.local/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/ziyuwang/.local/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 2.0531814098358154,
 'eval_rouge1': 21.6527,
 'eval_rouge2': 8.3782,
 'eval_rougeL': 18.1931,
 'eval_rougeLsum': 20.0461,
 'eval_gen_len': 19.0,
 'eval_runtime': 180.6774,
 'eval_samples_per_second': 9.016,
 'eval_steps_per_second': 0.565,
 'epoch': 5.0}

In [29]:
# tokenizer.save_pretrained()
# trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub("AlexWang99/flan-t5-base-A")

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-65f3faaf-50e57c7246cb89af4eb10e02;bd273732-e0a0-406b-bd3f-637c1175d6af)

Repository Not Found for url: https://huggingface.co/api/models/flan-t5-base-AlexWang99/MixSub.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [22]:
from transformers import pipeline
from random import randrange

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model="philschmid/flan-t5-base-samsum", device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-base summary:\n{res[0]['summary_text']}")

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-65f3f6f4-60b4910c7981d0760b1873a2;05c9f957-316f-4d14-a934-cb56e795d001)

Repository Not Found for url: https://huggingface.co/api/models/flan-t5-base-AlexWang99/MixSub.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [58]:
from transformers import AutoTokenizer
from random import randrange
import torch
sample = dataset['train'][1]
model = trainer.model
model.eval()
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
inputs = tokenizer(sample["Abstract"], return_tensors="pt", padding=True, truncation=True, max_length=2000)
inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")
with torch.no_grad():
    outputs = model.generate(**inputs,max_length=500)
print(f"prompt-abstract: \n{sample['Abstract']}\n---------------")
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"flan-t5-base highlights:\n{summary}")

prompt-abstract: 
Highlight: This study investigates consumer preference for sustainable food products and finds the classic phenomenon of preference reversal . Using contingent ranking and contingent valuation methods we investigate whether individuals rankings of a product are consistent with their willingness to pay . Results show that preference reversal exists for the evaluation of sustainable food products . Locally produced fresh produce receives the highest preference ranking while organic products receive the highest price premium . The incidence of preference reversal is not uniformly distributed among different groups of respondents . In general preference reversal is less frequent among respondents who are older and have a higher educational attainment . The results imply that preference reversal not only exists for subjects with explicit risks such as the lottery as indicated in the previous literature but also could occur for food products that consumers purchase on a reg

In [59]:
print(dataset['train'][1]['Highlights'])

@highlight Preference reversal exists in the context of sustainable food evaluation.
@highlight Contingent ranking and contingent valuation reveal a different preference.
@highlight A different preference reversal structure related to risks is found.
@highlight The level of inconsistency in preference is measured.
@highlight Older and higher educated respondents tend to have a more consistent preference.


In [16]:
from datasets import load_dataset, DatasetDict

# # Load dataset from the hub
dataset = load_dataset("AlexWang99/MixSub")

shuffled_dataset = dataset['train'].shuffle(seed=42) # Use a fixed seed for reproducibility

# Split the shuffled dataset into train, validation, and test sets (80%, 10%, 10%)
dataset_split = shuffled_dataset.train_test_split(test_size=0.2)
test_validation_split = dataset_split['test'].train_test_split(test_size=0.5)

train_set = dataset_split['train']
validation_set = test_validation_split['test']
test_set = test_validation_split['train']


# Add "highlight: " prefix to the 'abstract' field of each set
def add_prefix(examples):
    examples['Abstract'] = 'Generate Highlights for following text:' + examples['Abstract']
    return examples

train_set = train_set.map(add_prefix)
validation_set = validation_set.map(add_prefix)
test_set = test_set.map(add_prefix)
dataset = DatasetDict({
    'train': train_set,
    'test': test_set
})

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["Abstract", "Highlights"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Abstract"], truncation=True), batched=True, remove_columns=["Abstract", "Highlights"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Highlights"], truncation=True), batched=True, remove_columns=["Abstract", "Highlights"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["Abstract"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["Highlights"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


Map:   0%|          | 0/13035 [00:00<?, ? examples/s]

Map:   0%|          | 0/1629 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Map:   0%|          | 0/14664 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/14664 [00:00<?, ? examples/s]

Max target length: 352


In [17]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [18]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [19]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /home/ziyuwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
dataset_id="AlexWang99/MixSub"
# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-AlexWang99"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=3,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    generation_max_length = 50
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.306200,2.118653,35.006200,12.565000,26.692100,32.323200,48.995089
2,2.219800,2.098155,35.134600,12.601100,26.675900,32.435900,48.988950
3,2.185100,2.092047,35.510500,12.874300,26.840200,32.800700,48.988336


/home/ziyuwang/.local/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ziyuwang/.local/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=3261, training_loss=2.2484220699531394, metrics={'train_runtime': 5227.6424, 'train_samples_per_second': 7.48, 'train_steps_per_second': 0.624, 'total_flos': 2.677743626747904e+16, 'train_loss': 2.2484220699531394, 'epoch': 3.0})

In [26]:
tokenizer.save_pretrained()
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

TypeError: PreTrainedTokenizerBase.save_pretrained() missing 1 required positional argument: 'save_directory'

In [27]:
trainer.push_to_hub()

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-65f4c223-2dd1075645c971f33060741e;c2d16c3e-a291-414c-9abb-0af9b15ac35c)

Repository Not Found for url: https://huggingface.co/api/models/flan-t5-base-AlexWang99.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.